## <center>豆瓣网络数据分析</center>

### 由于网站反爬虫限制和网页的更改，可能会导致数据有差异，为了跟本章数据一致，请到本书提供的数据资源里下载db_data.txt。

# 12.1 数据处理
## （1）重新将已经保存好的数据db_data.txt读取到内存里进行数据处理。


In [ ]:
f1 = open(r'file/db_data.txt','r',encoding='utf-8')
f2 = f1.read()
type(f2)

In [ ]:
f2

In [ ]:
f2[:159]    #读取其中的前159个字符查看数据情况

读取到的数据f2是字符型，需要对数据进行转换，将f2转化为列表f3。

In [ ]:
f3 = eval(f2)     #还原到了data_all
type(f3)
f3[:5]

这里用到了eval()函数，将数据f2还原到了爬取数据时的data_all状态，即字典做成的列表。


## （2）将f3中的每一个元素（字典）中的值提取出来做成列表k，k中的每一个元素就是一个小说[名称，价格，星等级]的列表。

In [ ]:
k=[]
for i in f3:
    k.append(list(i.values()))

k[:10]#查看前10个元素

我们已经将从网上获取到的数据做成了列表，列表中的每个元数据就是一部小说的名称、价格、星等级3个数值做成的列表，即列表k中的每个元素还是列表。



## （3）将k列表做成一个数据框df，便于后面的数据清洗。

In [ ]:
import pandas as pd
df = pd.DataFrame(columns = ["title", "price", "star"])
p=0
for j in k:
    df.loc[p]=j
    p+=1
df.tail() #查看后5行数据

In [ ]:
df.to_excel(r'file/db_data.xlsx')#保存处理好的原数据


这里已经将数据处理成了数据框，并且查看数据框的最后5行数据，数据按照第一列为title、第二列为price、第三列为star排列，总数为1484条数据（包含了索引为0的数据）。


# 12.2 计算平均评分

我们已经将数据做成了数据框，星级数据在“star”列，可以使用df['star'].mean()计算star的平均星级，但运行df['star'].mean()时发现有错误提示。



In [ ]:
df['star'].mean()

从错误类型来看，主要是数据star列中的数据类型不全是float，也就是说star中含有str类型，即错误提示float和str不能相加。这就说明数据中有“异类”，要么是字符，要么是NaN，或者是其他的，总之不全是数值型。我们需要对这个“异类”进行排查。

In [ ]:
import pandas as pd
df['star']  = pd.to_numeric(df['star'],errors='coerce')#转成数值型，coerce表示无效数据设置成NaN
df['star'].astype(float).tail(50)

In [ ]:
df['star'].isnull().any()#对列判断，列有空或NAN元素就为True，否则为False

In [ ]:
df['star'][df['star'].isnull().values==True]#可以只显示存在缺失值的行列，清楚地确定缺失值的位置

发现有7个数据为缺失值NaN，为了方便数据处理，我们以0填充。


In [ ]:
df['star'] = df['star'].fillna(0)  #用填充空值，覆盖原df
df['star'][df['star'].isnull().values==True]#再次核查是否还有空缺值

In [ ]:
df['star'].mean()  #在没有空缺值的情况下再次计算“star”的均值

故第一问中所有爬取的小说的平均星级为8.327。  
说明如下。  
① 对于缺失数据一般的处理方法为滤掉或者填充。  
② 滤除缺失数据：dropna()。填充缺失数据：fillna()。  
③ 当只选择行里的数据全部为空才丢弃时，可向dropna()函数传入参数how='all'，如果以同样的方式按列丢弃，可以传入axis=1。


<pre>
1．用固定值填充
如果不想丢掉缺失的数据，而用默认值填充这些缺失值，可以使用fillna()函数，如df.fillna(0)；如果不想仅以某个标量填充，可以传入一个字典，如（fillna({})），对不同的列填充不同的值。
df.fillna({3:-1,2:100}) #第3列填充-1，第2列填充100

2．用均值填充
data_train.fillna(data_train.mean())  # 将所有行用各自的均值填充
data_train.fillna(data_train.mean()['browse_his', 'card_num']) # 也可以指定某些行进行填充

3．用上下数据进行填充
data_train.fillna(method='pad')  		#用前一个数据代替NaN：method='pad'
data_train.fillna(method='bfill')	#与pad相反，bfill表示用后一个数据代替
fillna()函数还有个参数limit，默认值None；如果指定了该参数，则连续的前向、后向填充NaN值的最大次数。换句话说，如果连续NaN数量超过这个数字，它将只被部分填充。如果未指定该参数，则沿着整个轴的NaN值被填充。
df.fillna(value=0, limit=3) #以0替换空值，并最多替换前3个
<code>

# 12.3 计算均价

再来看数据df的第二列price的数据情况。从数据的后5行来看，price列数据不整齐，有的带有单位元，属于字符型，为了计算小说的均价，需要处理掉汉字“元”，仅保留数字。



首先，大概浏览下数据概况。

In [ ]:
df['price'].head()

In [ ]:
df['price'].tail(15)

从数据的前5行和最后15行可以看出，数据不是很整齐，如29.00元、23.00、68、水如天儿等。为了发现更多的其他情况，继续查看中间的数据情况。



In [ ]:
df['price'][500:].head(15)          #查看第500行以后的开始15行数据

第503行数据为CNY 39.50。  
为了将price列处理为数值，需要将price数据的前后非数值字符处理掉。


In [ ]:
df_rstrip = df['price'].str.rstrip('元')

In [ ]:
df_rstrip.head()

In [ ]:
df_rstrip.tail(10)

In [ ]:
df_rstrip[503]

In [ ]:
df['price']= df_rstrip

为了去掉左边的非数字的字符，需要将空缺值找出来并赋值为0。

In [ ]:
a = df[df['price'].isin([''])].index.tolist()   #从price列中定位给定值，即找出空缺值的位置，并给出这些值的索引列表
print(a)

可以看出空缺值仅有索引为1160的行。对1160行的缺失值赋值为0。



In [ ]:
df.loc[1160,'price']=0
df.iloc[1160] #查看1160行的数据

我们先写一个函数，功能是删除给定的字符串，删除其左边的非数字的字符，如果全部为非数字的字符，则将此字符串赋值0，如果字符串全部为数字，则字符串不变。

In [ ]:
df['price'].tail(50)

In [ ]:
df['price'].count()

In [ ]:
def del_l_str(txt):
     '''
     delte arg's left_string
     删除数据的左侧非数字，当全部为非数字时，返回0
     输入只能是字符型，若输入为空，则返回错误
     '''
     strings = str(txt)
     j = 0
     while not strings[j].isdigit(): #判断数据中的第j个字符是不是数字
#          print(strings[j])
         j += 1
         if j == len(strings):
             break
         #print(j)
     if 0 < j < len(strings):
         r = strings[j:] 
     else:
         r = 0
     return r  


使用del_l_str()函数删除price列每个数据左侧的非数字字符。

In [ ]:
import copy
print(len(df['price']) )
print(df['price'])
df_copy = copy.deepcopy(df)  #######################
n = 0 #标记索引号
for k in df['price']: 
    df_copy['price'][n] = del_l_str(k)
    n += 1

In [ ]:
df_copy['price'].tail(50)#查看最后50个数据

通过删除数据的左右非数字字符，绝大部分数据已经被处理成了纯数字，但为了防止数据中还有其他的“杂质”，强制将其他非数字行转化为nan，再将nan替换成数值0。



In [ ]:
df_copy['price'] = pd.to_numeric(df_copy['price'],errors='coerce')
c = df_copy[df_copy['price'].isin(['NaN'])].index.tolist()#找出空缺值的位置并给出其索引列表
print(c)

将这些强制转化为空的数据替换成0。

In [ ]:
for i in c:
    df_copy.loc[i,'price'] = 0
df_copy.iloc[697] #查看索引为697行的数据

至此数据的处理已经完成。为了查验数据是否缺项，可以先用count()函数进行统计，再求均值。

In [ ]:
df_copy['price'].count()

In [ ]:
df_copy['price'].tail(50)

In [ ]:
str(df_copy['price'].mean())

说明：数据已经处理完毕，均值已经计算出来，但是这样处理数据还不是很合理。例如，数据的单位不统一，有台币、人民币，还有美元，应该先进行相应的单位换算，再计算均值，或者将这些行删除后，再进行均值计算。这些留给读者思考并自行完成。